In [1]:
def create_game_info(url, season_id, season_gamecount):
    
    game_count = str(season_gamecount)
    while len(game_count) < 4:
        game_count = '0' + game_count
    
    id_string = url.strip(string.ascii_letters+string.punctuation)
    year = id_string[0:4]
    month = id_string[4:6]
    day = id_string[6:8]
    
    date = year+'-'+month+'-'+day
    
    game_id = int(season_id+month+day+game_count)
    season_id = int(season_id)
    
    return [game_id, season_id, date]

In [2]:
def create_team_info(table):
    '''
    Create a dataframe with game results. Uses an html table as input.
    
    ---
    Inputs:
    
    table: a BeautifulSoup html table
    ---
    Outputs:
    
    team_info: a dataframe with the relevant game information (team_ids, scores, and boolean 'results' column)
    '''
    
    # get team_ids
    id_rows = table.findAll('th', attrs={'class':'center', 'data-stat':'team', 'scope':'row'})
    team_ids = [row.text.strip() for row in id_rows]
    
    # get final score
    scores = table.findAll('td', attrs={'class': 'center', 'data-stat': 'T'})
    final_scores = [int(score.text.strip()) for score in scores]
    
    # boolean game-winner: away=0, home=1
    if final_scores[0] > final_scores[1]:
        result=0
    else:
        result=1
    
    team_info = [team_ids[0], final_scores[0], team_ids[1], final_scores[1], result]
    
    return team_info

In [3]:
def create_info_df(game_info, team_info, info_columns):
    info = game_info + team_info
    info_df = pd.DataFrame([info], columns=info_columns)
    return info_df

In [4]:
def create_boxscores(table, game_id):

    # ignore first 'tr', it is table title, not column
    rows = table.findAll('tr')[1:]
    # first 'th' is 'Starters', but will be changed into the player names
    headers = rows[0].findAll('th')
    # provide column names
    headerlist = [h.text.strip() for h in headers]
    
    # ignore first row (headers)
    data = rows[1:]
    # get names column
    player_names = [row.find('th').text.strip() for row in rows]
    # get player stats
    player_stats = [[stat.text.strip() for stat in row.findAll('td')] for row in data]
    # add player name as first entry in each row
    for i in range(len(player_stats)):
        # ignore header with i+1
        player_stats[i].insert(0, player_names[i+1])
    
    # create player stats dataframe
    player_box_df = pd.DataFrame(player_stats, columns=headerlist)
    # drop 'Reserves' row
    player_box_df.drop(player_box_df[player_box_df['Starters'] == 'Reserves'].index, inplace=True)
    
    # add game id column
    player_box_df.insert(loc=0, column='game_id', value=game_id)
    
    # create team stats dataframe from last row in player stats
    team_box_df = pd.DataFrame(player_box_df.iloc[-1]).T
    
    #drop team totals from player stats df
    player_box_df = player_box_df[:-1].rename(columns={'Starters': 'player'})

    return player_box_df, team_box_df

In [5]:
def merge_boxscores(boxscore_list, team_ids, scope):

    # create tuple for every 2 boxscores in list
    pairs = [((boxscore_list[i]), (boxscore_list[i + 1])) for i in range(0, len(boxscore_list), 2)]
    
    clean_boxscores= []
    
    for pair in pairs:
        
        # combine regular and adv boxscores
        df = pd.concat([*pair], axis=1)
        # drop columns with duplicate names
        df = df.loc[:,~df.columns.duplicated()].copy()
        
        clean_boxscores.append(df)
    
    for i in range(len(clean_boxscores)):
        
        if scope=='team':
            clean_boxscores[i].rename(columns={'Starters': 'team'}, inplace=True)
            clean_boxscores[i]['team'] = team_ids[i]
            
        elif scope=='player':
            clean_boxscores[i].insert(loc=2, column='team', value=team_ids[i])
    
    return clean_boxscores

In [6]:
def change_dtypes(df, num_columns):

    df.replace(to_replace='', value='-99', inplace=True)
    
    for column in num_columns:
        df[column] = df[column].astype('float64')
        
    df.replace(to_replace=-99, value=np.nan, inplace=True)
    
    return df

In [7]:
def create_PIE(player_boxes, totals):
    
    PIE_denom = (totals['PTS'] + totals['FG'] + totals['FT'] - totals['FGA'] - totals['FTA'] + totals['DRB'] + (0.5*totals['ORB']) + totals['AST'] + totals['STL'] + (0.5*totals['BLK']) - totals['PF'] - totals['TOV'])
    player_boxes['PIE'] = round((100 * (player_boxes['PTS'] + player_boxes['FG'] + player_boxes['FT'] - player_boxes['FGA'] - player_boxes['FTA'] + player_boxes['DRB'] + (0.5*player_boxes['ORB']) + player_boxes['AST'] + player_boxes['STL'] + (0.5*player_boxes['BLK']) - player_boxes['PF'] - player_boxes['TOV']) / PIE_denom), 1)
    
    return player_boxes

In [8]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import string
import time
import timeit
import sqlite3

pd.set_option('display.max_columns', None)

In [9]:
# global variables


base_url = 'https://www.basketball-reference.com'
url = r'https://www.basketball-reference.com/leagues/NBA_2018_games.html'

season_id = '1718'
season_gamecount = 1

precovid_seasons = ['1314', '1415', '1516', '1617', '1718', '1819']
postcovid_seasons = ['1920', '2021', '2122', '2223']

last_game_url = {'1920': base_url+'/boxscores/202008140HOU.html',
                 '2021': base_url+'/boxscores/202105160SAC.html',
                 '2122': base_url+'/boxscores/202204100POR.html',
                 '2223': base_url+'/boxscores/202304090POR.html'}
last_game_row = {'1920': 83,
                 '2021': 140,
                 '2122': 83,
                 '2223': 72}

info_columns = ['game_id', 'season', 'date', 'away_team', 'away_score', 'home_team', 'home_score', 'result']
num_columns = ['FG', 'FGA', '3P', '3PA', 'FT', 'FTA', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '+/-',
               'FG%', '3P%', 'FT%', 'TS%', 'eFG%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'ORtg', 'DRtg', 'BPM']

In [10]:
driver = webdriver.Chrome()

In [17]:
# ping1
driver.get(url)

## get boxscore urls for october

In [26]:
src = driver.page_source
parser = BeautifulSoup(src, 'lxml')
table = parser.find('div', attrs = {'class': 'table_container is_setup'})
headers = table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'})
oct_17_18_url_list = [base_url + header.a['href'] for header in headers]

In [32]:
# splits = table.findAll('tr', attrs = {'class': 'thead'})
# for split in splits:
#     if 'Playoffs' in split.text:
#         row_num = int(split['data-row'])

## get individual boxscores and game info:

In [ ]:
conn = sqlite3.connect('test_database')
# c = conn.cursor()

start_time = time.time()


for i in range(2):
    
    # ping2
    driver.get(oct_17_18_url_list[i])

    src = driver.page_source
    parser = BeautifulSoup(src, 'lxml')

    stat_tables = parser.findAll('table', attrs = {'class': 'sortable stats_table now_sortable'})

    id_table = parser.find('table', attrs = {'class': 'suppress_all stats_table', 'id': 'line_score'})

    game_info = create_game_info(url=oct_17_18_url_list[i],
                                 season_id=season_id,
                                 season_gamecount=season_gamecount)
    game_id = game_info[0]

    team_info = create_team_info(id_table)

    team_ids = [team_info[0], team_info[2]]

    info_df = create_info_df(game_info=game_info,
                             team_info=team_info,
                             info_columns=info_columns)

    player_box_list = [None, None, None, None]
    team_box_list = [None, None, None, None]

    for i in range(len(stat_tables)):
        player_box_list[i], team_box_list[i] = create_boxscores(stat_tables[i], game_id=game_id)

    away_team_box, home_team_box = merge_boxscores(team_box_list, team_ids=team_ids, scope='team')

    team_boxes = pd.concat([away_team_box, home_team_box])
    team_boxes.reset_index(drop=True, inplace=True)

    team_boxes = change_dtypes(team_boxes, num_columns)

    away_player_box, home_player_box = merge_boxscores(player_box_list, team_ids=team_ids, scope='player')

    player_boxes = pd.concat([away_player_box, home_player_box])
    player_boxes.reset_index(drop=True, inplace=True)

    player_boxes = change_dtypes(player_boxes, num_columns)

    totals = dict(team_boxes.loc[:,'FG':'PTS'].sum())

    player_boxes = create_PIE(player_boxes, totals)

    info_df.to_sql('game_info', con=conn, if_exists='append', index=False)
    team_boxes.to_sql('team_stats', con=conn, if_exists='append', index=False)
    player_boxes.to_sql('player_stats', con=conn, if_exists='append', index=False)
    
    season_gamecount += 1
    
conn.close()

end_time = time.time()

duration = end_time-start_time

In [ ]:
duration

In [ ]:
conn = sqlite3.connect('test_database')

In [ ]:
query = "SELECT * from game_info"
df = pd.read_sql_query(query, conn)

In [ ]:
df

## get all month links for a single season

In [11]:
driver.get(url)

In [12]:
# ping 2
src = driver.page_source
parser = BeautifulSoup(src, 'lxml')
months = parser.find('div', attrs = {'class': 'filter'})
links = months.findAll('a')
month_links = [base_url + link['href'] for link in links]

## how to limit it to only the regular season months?

In [13]:
month_links = month_links[0:7]

In [14]:
driver.get(month_links[-1])

## for pre-covid seasons: get only the reg season games in final month

In [15]:
src = driver.page_source
parser = BeautifulSoup(src, 'lxml')
table = parser.find('div', attrs = {'class': 'table_container is_setup'})
header_rows = table.findAll('tr', attrs = {'class': 'thead'})

In [16]:
header_rows

[<tr class="thead" data-row="88"><th colspan="11">Playoffs</th></tr>]

In [ ]:
if header_rows[0].text == 'Playoffs':
    row_num = int(header_rows[0]['data-row'])

In [ ]:
int(header_rows[0]['data-row'])

In [ ]:
headers = table.findAll('td', attrs = {'class': 'center', 'data-stat': 'box_score_text'}, limit=88)
# apr_17_18_url_list = [base_url + header.a['href'] for header in headers]

In [ ]:
headers

## for post covid, i need to use the hard coded values for last reg season game each season

In [ ]:

# links[0]['href']

# driver.find_element(By.LINK_TEXT, 'October')

# driver.find_element(By.LINK_TEXT, 'October').click();

# months = ['October', 'November', 'December', 'January', 'February', 'March', 'April']

In [ ]:
# np.random.randint(5,16)

In [ ]:
# conn = sqlite3.connect('test_database')
# c = conn.cursor()

# away_player_box.to_sql('player_stats', con=conn, if_exists='append', index=False)

# home_player_box.to_sql('player_stats', con=conn, if_exists='append', index=False)

In [ ]:
query = "SELECT * from player_stats"

# pd.read_sql_query(query, conn, index_col='index')
# df.index.names = ['']

In [ ]:
dffd = pd.read_sql_query(query, conn, index_col='index')
dffd

In [ ]:
for season in precovid_seasons:
    

In [ ]:
conn.close()

In [31]:
driver.quit()

In [ ]:
# select = Select(driver.find_element_by_xpath(r'htmlcodehere'))

# select.select_by_index(0)         # selects 0th option from the drop down menu of this xpathed element